In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T
import torch.nn.functional as F

import numpy as np

USE_GPU = True
dtype = torch.float32

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print_every = 100
print('using device:', device)

using device: cuda


In [2]:
NUM_TRAIN = 49000

# preprocess data by subtracting mean RGB and dividing by std (hardcoded)
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Dataset object for each split (train / val / test)
# Datasets load examples one at a time, so wrap in Dataloader which iterates through Dataset to form minibatches
# Divide CIFAR-10 training set into train and val by passing Sampler object telling it how it should sample from underlying Dataset
cifar10_train = dset.CIFAR10('./datasets', train=True, download=True, transform=transform)
loader_train = DataLoader(cifar10_train, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar10_val = dset.CIFAR10('./datasets', train=True, download=True, transform=transform)
loader_val = DataLoader(cifar10_val, batch_size=64, sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar10_test = dset.CIFAR10('./datasets', train=False, download=True, transform=transform)
loader_test = DataLoader(cifar10_test, batch_size=64)

100.0%


In [7]:
test = torch.Tensor(([1,2,3], [4,5,6]))
print(test)
print(torch.flatten(test))
print(test.dtype)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([1., 2., 3., 4., 5., 6.])
torch.float32


In [ ]:
# Input img size: 32 x 32 x 3

# [CONV -> ReLU] -> [CONV -> ReLU] -> [FC]
# channel_1: 5x5 filters, zero_padding of 2
# channel_2: 3x3 filters, zero_padding of 1

class ThreeLayerConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channel, channel_1, 5, padding=2)
        nn.init.kaiming_normal_(self.conv1.weight)

        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)

        self.fc = nn.Linear(channel_2 * 32 * 32, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = x.view(x.shape[0], -1)
        scores = self.fc(x)
        return scores
    
def test_ThreeLayerConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)
    model = ThreeLayerConvNet(in_channel=3, channel_1=12, channel_2=8, num_classes=10)
    scores = model(x)
    print(scores.shape)

test_ThreeLayerConvNet()

torch.Size([64, 10])


In [16]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')

    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=dtype)
            scores = model(x)
            _, preds = scores.max(dim=1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [19]:
def train(model, optimizer, epochs=1):
    model = model.to(device=device)
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()
            x = x.to(device=device, dtype=dtype)
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy(loader_val, model)
                print()

In [21]:
learning_rate = 1e-3
channel_1 = 32
channel_2 = 16
num_classes = 10

model = ThreeLayerConvNet(3, channel_1, channel_2, num_classes)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Iteration 0, loss = 2.8570
Checking accuracy on validation set
Got 125 / 1000 correct (12.50)

Iteration 100, loss = 1.8136
Checking accuracy on validation set
Got 376 / 1000 correct (37.60)

Iteration 200, loss = 1.2723
Checking accuracy on validation set
Got 467 / 1000 correct (46.70)

Iteration 300, loss = 1.3818
Checking accuracy on validation set
Got 498 / 1000 correct (49.80)

Iteration 400, loss = 1.5267
Checking accuracy on validation set
Got 532 / 1000 correct (53.20)

Iteration 500, loss = 1.2989
Checking accuracy on validation set
Got 564 / 1000 correct (56.40)

Iteration 600, loss = 1.3560
Checking accuracy on validation set
Got 540 / 1000 correct (54.00)

Iteration 700, loss = 1.0702
Checking accuracy on validation set
Got 580 / 1000 correct (58.00)

Iteration 0, loss = 1.0862
Checking accuracy on validation set
Got 596 / 1000 correct (59.60)

Iteration 100, loss = 0.8761
Checking accuracy on validation set
Got 583 / 1000 correct (58.30)

Iteration 200, loss = 0.7956
Check

In [26]:
class ConvNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, channel_4, num_classes):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channel, channel_1, 3, padding=1)
        nn.init.kaiming_normal_(self.conv1.weight)

        self.conv2 = nn.Conv2d(channel_1, channel_2, 3, padding=1)
        nn.init.kaiming_normal_(self.conv2.weight)

        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(channel_2, channel_3, 3, padding=1)
        nn.init.kaiming_normal_(self.conv3.weight)

        self.conv4 = nn.Conv2d(channel_3, channel_4, 3, padding=1)
        nn.init.kaiming_normal_(self.conv4.weight)

        self.pool2 = nn.MaxPool2d(2, 2)

        self.fc = nn.Linear(channel_4 * 8 * 8, num_classes)
        nn.init.kaiming_normal_(self.fc.weight)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = x.view(x.shape[0], -1)
        scores = self.fc(x)
        return scores
    
def test_ConvNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)
    model = ConvNet(in_channel=3, channel_1=32, channel_2=32, channel_3=32, channel_4=32, num_classes=10)
    scores = model(x)
    print(scores.shape)

test_ConvNet()

torch.Size([64, 10])


In [24]:
learning_rate = 1e-3
channel_1 = 32
channel_2 = 32
channel_3 = 32
channel_4 = 32
num_classes = 10

model = ConvNet(3, channel_1, channel_2, channel_3, channel_4, num_classes)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Iteration 0, loss = 2.7879
Checking accuracy on validation set
Got 137 / 1000 correct (13.70)

Iteration 100, loss = 1.5430
Checking accuracy on validation set
Got 430 / 1000 correct (43.00)

Iteration 200, loss = 1.5208
Checking accuracy on validation set
Got 502 / 1000 correct (50.20)

Iteration 300, loss = 1.3502
Checking accuracy on validation set
Got 507 / 1000 correct (50.70)

Iteration 400, loss = 1.2136
Checking accuracy on validation set
Got 540 / 1000 correct (54.00)

Iteration 500, loss = 1.2181
Checking accuracy on validation set
Got 573 / 1000 correct (57.30)

Iteration 600, loss = 1.1822
Checking accuracy on validation set
Got 586 / 1000 correct (58.60)

Iteration 700, loss = 1.4702
Checking accuracy on validation set
Got 603 / 1000 correct (60.30)

Iteration 0, loss = 1.0568
Checking accuracy on validation set
Got 618 / 1000 correct (61.80)

Iteration 100, loss = 0.9796
Checking accuracy on validation set
Got 643 / 1000 correct (64.30)

Iteration 200, loss = 0.9742
Check

In [25]:
check_accuracy(loader_test, model)

Checking accuracy on test set
Got 7259 / 10000 correct (72.59)


In [39]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channel, out_channel, 3, padding=1)
        self.bn = nn.BatchNorm2d(out_channel)
        self.conv2 = nn.Conv2d(out_channel, out_channel, 3, padding=1)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn(out)
        out = F.relu(out)
        out = self.conv2(out)

        out += identity
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, channel_3, channel_4, channel_5, num_classes):
        super().__init__()

        self.conv = nn.Conv2d(in_channel, channel_1, 3, padding=1)
        self.block1 = ResidualBlock(channel_1, channel_2)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.block2 = ResidualBlock(channel_3, channel_4)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(channel_4 * 8 * 8, channel_5)
        self.fc2 = nn.Linear(channel_5, num_classes)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.block1(x)
        x = self.pool1(x)
        x = self.block2(x)
        x = self.pool2(x)
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        scores = self.fc2(x)
        return scores
    
def test_ResNet():
    x = torch.zeros((64, 3, 32, 32), dtype=dtype)
    model = ResNet(in_channel=3, channel_1=32, channel_2=32, channel_3=32, channel_4=32, channel_5=32, num_classes=10)
    scores = model(x)
    print(scores.shape)

test_ResNet()

torch.Size([64, 10])


In [40]:
learning_rate = 1e-3
channel_1 = 64
channel_2 = 64
channel_3 = 64
channel_4 = 64
channel_5 = 64
num_classes = 10

model = ResNet(3, channel_1, channel_2, channel_3, channel_4, channel_5, num_classes)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

train(model, optimizer, epochs=10)

Iteration 0, loss = 2.3899
Checking accuracy on validation set
Got 152 / 1000 correct (15.20)

Iteration 100, loss = 1.7146
Checking accuracy on validation set
Got 408 / 1000 correct (40.80)

Iteration 200, loss = 1.4834
Checking accuracy on validation set
Got 497 / 1000 correct (49.70)

Iteration 300, loss = 1.5038
Checking accuracy on validation set
Got 519 / 1000 correct (51.90)

Iteration 400, loss = 1.2709
Checking accuracy on validation set
Got 575 / 1000 correct (57.50)

Iteration 500, loss = 0.9742
Checking accuracy on validation set
Got 601 / 1000 correct (60.10)

Iteration 600, loss = 0.9720
Checking accuracy on validation set
Got 629 / 1000 correct (62.90)

Iteration 700, loss = 0.9465
Checking accuracy on validation set
Got 642 / 1000 correct (64.20)

Iteration 0, loss = 1.1325
Checking accuracy on validation set
Got 605 / 1000 correct (60.50)

Iteration 100, loss = 0.9750
Checking accuracy on validation set
Got 647 / 1000 correct (64.70)

Iteration 200, loss = 0.8180
Check

In [41]:
check_accuracy(loader_test, model)

Checking accuracy on test set
Got 7546 / 10000 correct (75.46)
